# Setup

In [ ]:
%pip install torch-geometric
%pip install pynvml

In [1]:
import os
import math
import time
import copy
import psutil
import pynvml
import random
import numpy as np
import networkx as nx

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler, random_split

import torch_geometric.utils as utils
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix

from scipy.sparse import csgraph
from scipy.sparse.linalg import eigsh
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
print(f'using {device}')

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

set_seed(42)

using cpu


# Data Preprocessing

In [24]:
dataset_path = '/notebooks/dataset/'


def data_cleansing(dataset):
    # Replace negative values with 0
    dataset[dataset < 0] = 0
    
    # Replace NaN values with 0
    dataset = np.nan_to_num(dataset, nan=0)
    
    return dataset

def check_and_drop_invalid_graphs(graph_dataset):
    num_graphs, num_timepoints, num_nodes, _ = graph_dataset.shape
    num_dimensions = 1
    
    valid_graphs = []

    for i in range(num_graphs):
        is_valid = True
        for t in range(num_timepoints):
            adj_matrix = graph_dataset[i, t, :, :]
            num_edges = np.sum(adj_matrix > 0)
            if num_edges == 0:
                is_valid = False
                break
        
        if is_valid:
            valid_graphs.append(i)
    
    cleaned_dataset = graph_dataset[valid_graphs, :, :, :]
    
    return cleaned_dataset

def convert_to_pyg_data(adj_matrices, inp_features):
    pyg_data_list = []
    
    num_graphs = adj_matrices.shape[0]
    num_timepoints = adj_matrices.shape[1]
    
    for i in range(num_graphs):
        for t in range(num_timepoints):
            adj_matrix = adj_matrices[i, t]
            features = inp_features[i, t]
            
            # Get edge indices
            edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
            
            # Get edge attributes
            edge_attr = torch.tensor(adj_matrix[adj_matrix.nonzero()], dtype=torch.float)
            
            # Node features
            x = torch.tensor(features, dtype=torch.float)
            
            # Label (timepoint)
            y = torch.tensor([t], dtype=torch.long)
            
            pyg_data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
            pyg_data_list.append(pyg_data)
    
    return pyg_data_list

def get_dataset_info(pyg_data_list):
    num_features = pyg_data_list[0].x.shape[1]
    all_labels = torch.cat([data.y for data in pyg_data_list])
    num_classes = torch.unique(all_labels).numel()
    return num_features, num_classes

def create_data_loaders(pyg_data_list, batch_size, seed=42):
    set_seed(seed)
    train_val_data, test_data = train_test_split(pyg_data_list, test_size=0.2, random_state=seed)
    train_data, val_data = train_test_split(train_val_data, test_size=0.1, random_state=seed)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

def process_connectomic_dataset(dataset_name, adj, features, batch_size=32):
    print(f'\n{dataset_name} Dataset')
    pyg_data_list = convert_to_pyg_data(adj, features)
    print(pyg_data_list[0])
    
    num_features, num_classes = get_dataset_info(pyg_data_list)
    print(f'Num Features = {num_features}, Num Classes = {num_classes}')
    
    return pyg_data_list, num_features, num_classes

In [12]:
simulated = np.load(dataset_path + 'simulated_adj.npy')
simulated_cleaned = data_cleansing(simulated[:,:,:,:,0])
simulated_adj = check_and_drop_invalid_graphs(simulated_cleaned)

## EMCI-AD Dataset
emci = np.load(dataset_path + 'emci-ad_adj.npy')
emci_cleaned = data_cleansing(emci)
emci_adj = check_and_drop_invalid_graphs(emci_cleaned)

## SLIM160 Dataset
slim160 = np.load(dataset_path + 'slim160_adj.npy')
slim160_cleaned = data_cleansing(slim160)
slim160_adj = check_and_drop_invalid_graphs(slim160_cleaned)

## Node Features Initialization

In [10]:
def initialize_with_ones(graphs, feature_dim=8):
    """
    Initialize node features with ones.
    
    :param graphs: numpy array of shape [n_subjects, n_timepoints, n_nodes, n_nodes]
    :param feature_dim: Dimension of the node features
    :return: numpy array of shape [n_subjects, n_timepoints, n_nodes, feature_dim]
    """
    n_subjects, n_timepoints, n_nodes, _ = graphs.shape
    node_features = np.ones((n_subjects, n_timepoints, n_nodes, feature_dim))
    
    return node_features

def initialize_with_random(graphs, feature_dim=8):
    """
    Initialize node features with random values.
    
    :param graphs: numpy array of shape [n_subjects, n_timepoints, n_nodes, n_nodes]
    :param feature_dim: Dimension of the node features
    :return: numpy array of shape [n_subjects, n_timepoints, n_nodes, feature_dim]
    """
    n_subjects, n_timepoints, n_nodes, _ = graphs.shape
    node_features = np.random.rand(n_subjects, n_timepoints, n_nodes, feature_dim)
    
    return node_features

def laplacian_positional_encoding(adj_matrix, dim, ncv=None):
    laplacian = csgraph.laplacian(adj_matrix, normed=True)
    try:
        eigvals, eigvecs = eigsh(laplacian, k=dim + 1, which='SM', ncv=ncv)
        encoding = torch.tensor(eigvecs[:, 1:], dtype=torch.float)  # Skip the first eigenvector
    except Exception as e:
        print(f"Error for matrix with shape {adj_matrix.shape}: {e}")
        encoding = torch.zeros((adj_matrix.shape[0], dim), dtype=torch.float)  # Fallback to zero encoding
    return encoding

def process_graph(args):
    i, t, graphs, num_nodes, feature_dim = args
    adj_matrix = graphs[i, t, :, :]
    return (i, t, laplacian_positional_encoding(adj_matrix, feature_dim).numpy())

def laplacian_initialization(graphs, feature_dim=8):
    """
    Initialize node features using Laplacian positional encoding.
    
    :param graphs: numpy array of shape [n_subjects, n_timepoints, n_nodes, n_nodes]
    :param feature_dim: Dimension of the node features
    :return: numpy array of shape [n_subjects, n_timepoints, n_nodes, feature_dim]
    """
    n_subjects, n_timepoints, n_nodes, _ = graphs.shape
    node_features = np.zeros((n_subjects, n_timepoints, n_nodes, feature_dim))
    
    tasks = [(i, t, graphs, n_nodes, feature_dim) for i in range(n_subjects) for t in range(n_timepoints)]
    with ThreadPoolExecutor() as executor:
        for i, t, encoding in tqdm(executor.map(process_graph, tasks), total=len(tasks)):
            node_features[i, t, :, :] = encoding

    return node_features

In [17]:
simulated_ones_features = initialize_with_ones(simulated_adj)
simulated_random_features = initialize_with_random(simulated_adj)
simulated_laplacian_features = laplacian_initialization(simulated_adj)
print(f'SIMULATED -> ones shape: {simulated_ones_features.shape}, random shape:{simulated_random_features.shape}, laplacian shape:{simulated_laplacian_features.shape}')

emci_ones_features = initialize_with_ones(emci_adj)
emci_random_features = initialize_with_random(emci_adj)
emci_laplacian_features = laplacian_initialization(emci_adj)
print(f'EMCI-AD -> ones shape: {emci_ones_features.shape}, random shape:{emci_random_features.shape}, laplacian shape:{emci_laplacian_features.shape}')

slim160_ones_features = initialize_with_ones(slim160_adj)
slim160_random_features = initialize_with_random(slim160_adj)
slim160_laplacian_features = laplacian_initialization(slim160_adj)
print(f'SLIM160 -> ones shape: {slim160_ones_features.shape}, random shape:{slim160_random_features.shape}, laplacian shape:{slim160_laplacian_features.shape}')

100%|██████████| 300/300 [00:00<00:00, 604.30it/s]


SIMULATED -> ones shape: (100, 3, 35, 8), random shape:(100, 3, 35, 8), laplacian shape:(100, 3, 35, 8)


100%|██████████| 134/134 [00:00<00:00, 136.03it/s]


EMCI-AD -> ones shape: (67, 2, 35, 8), random shape:(67, 2, 35, 8), laplacian shape:(67, 2, 35, 8)


100%|██████████| 327/327 [00:00<00:00, 344.79it/s]

SLIM160 -> ones shape: (109, 3, 160, 8), random shape:(109, 3, 160, 8), laplacian shape:(109, 3, 160, 8)


## Data Construction

In [43]:
## Simulated dataset
simulated_ones, simulated_ones_num_features, simulated_ones_num_classes = process_connectomic_dataset(
    'Simulated-ones', simulated_adj, simulated_ones_features)
simulated_random, simulated_random_num_features, simulated_random_num_classes = process_connectomic_dataset(
    'Simulated-random', simulated_adj, simulated_random_features)
simulated_laplacian, simulated_laplacian_num_features, simulated_laplacian_num_classes = process_connectomic_dataset(
    'Simulated-laplacian', simulated_adj, simulated_laplacian_features)

## EMCI-AD dataset
emci_ones, emci_ones_num_features, emci_ones_num_classes = process_connectomic_dataset(
    'EMCI-AD-ones', emci_adj, emci_ones_features)
emci_random, emci_random_num_features, emci_random_num_classes = process_connectomic_dataset(
    'EMCI-AD-random', emci_adj, emci_random_features)
emci_laplacian, emci_laplacian_num_features, emci_laplacian_num_classes = process_connectomic_dataset(
    'EMCI-AD-laplacian', emci_adj, emci_laplacian_features)

## slim160 dataset
slim160_ones, slim160_ones_num_features, slim160_ones_num_classes = process_connectomic_dataset(
    'SLIM160-ones', slim160_adj, slim160_ones_features, batch_size=1)
slim160_random, slim160_random_num_features, slim160_random_num_classes = process_connectomic_dataset(
    'SLIM160-random', slim160_adj, slim160_random_features)
slim160_laplacian, slim160_laplacian_num_features, slim160_laplacian_num_classes = process_connectomic_dataset(
    'SLIM160-laplacian', slim160_adj, slim160_laplacian_features)


Simulated-ones Dataset
Data(x=[35, 8], edge_index=[2, 1168], edge_attr=[1168], y=[1])
Num Features = 8, Num Classes = 3

Simulated-random Dataset
Data(x=[35, 8], edge_index=[2, 1168], edge_attr=[1168], y=[1])
Num Features = 8, Num Classes = 3

Simulated-laplacian Dataset
Data(x=[35, 8], edge_index=[2, 1168], edge_attr=[1168], y=[1])
Num Features = 8, Num Classes = 3

EMCI-AD-ones Dataset
Data(x=[35, 8], edge_index=[2, 1178], edge_attr=[1178], y=[1])
Num Features = 8, Num Classes = 2

EMCI-AD-random Dataset
Data(x=[35, 8], edge_index=[2, 1178], edge_attr=[1178], y=[1])
Num Features = 8, Num Classes = 2

EMCI-AD-laplacian Dataset
Data(x=[35, 8], edge_index=[2, 1178], edge_attr=[1178], y=[1])
Num Features = 8, Num Classes = 2

SLIM160-ones Dataset
Data(x=[160, 8], edge_index=[2, 19994], edge_attr=[19994], y=[1])
Num Features = 8, Num Classes = 3

SLIM160-random Dataset
Data(x=[160, 8], edge_index=[2, 19994], edge_attr=[19994], y=[1])
Num Features = 8, Num Classes = 3

SLIM160-laplacian D

# Models

In [5]:
## GCN Models
class GCN(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GCN, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters() 

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class GCN1Layer(torch.nn.Module):
    def __init__(self, num_features, hidden_features, num_classes):
        super(GCN1Layer, self).__init__()
        self.gcn1 = GCN(num_features, hidden_features)
        self.bn1 = nn.BatchNorm1d(hidden_features)

        self.fc = torch.nn.Linear(hidden_features, num_classes)

    def forward(self, x, adj, batch):
        x = F.relu(self.bn1(self.gcn1(x, adj)))
        x = global_mean_pool(x, batch)  
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

class GCN2Layer(torch.nn.Module):
    def __init__(self, num_features, hidden_features, num_classes):
        super(GCN2Layer, self).__init__()
        self.gcn1 = GCN(num_features, hidden_features)
        self.bn1 = nn.BatchNorm1d(hidden_features)
        self.gcn2 = GCN(hidden_features, hidden_features*2)
        self.bn2 = nn.BatchNorm1d(hidden_features*2)

        self.fc = torch.nn.Linear(hidden_features*2, num_classes)

    def forward(self, x, adj, batch):
        x = F.relu(self.bn1(self.gcn1(x, adj)))
        x = F.relu(self.bn2(self.gcn2(x, adj)))
        x = global_mean_pool(x, batch)  # Global mean pooling
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad) 

# Trainings & Evaluations

In [6]:
## Single Functions
def single_train(model, loader, val_loader, lr=0.001, num_epochs=100, patience=5, 
                step_size=50, gamma=0.5, save_path='models/gcn_x.pth', 
                binary_classification=True, is_esn=False):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.NLLLoss()
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    if is_esn == True:
        spectral_hook = SpectralRadiusOptimizerHook(model)

    training_loss = []
    validation_loss = []
    epoch_time = []
    cpu_usage_percent = []
    memory_usage = []
    gpu_usage = []
    gpu_usage_percent = []

    best_train_loss = float('inf')
    best_val_loss = float('inf')
    epochs_no_improve = 0

    model.to(device)

    # Get the process object for the current process
    process = psutil.Process()

    for epoch in range(num_epochs):
        epoch_loss = 0

        set_seed(42)
        model.train()
        epoch_start_time = time.time()

        # Measure CPU and GPU usage before the epoch
        cpu_usage_before = psutil.cpu_percent(interval=None)
        memory_before = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_before = torch.cuda.memory_allocated(device)
            gpu_util_before = torch.cuda.utilization(device)
        else:
            gpu_usage_before = 0
            gpu_util_before = 0

        for data in loader:
            x, edge_index, batch, y = data.x.to(device), data.edge_index.to(device), data.batch.to(device), data.y.to(device)
            adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)

            optimizer.zero_grad()
            output = model(x, adj_matrix, batch)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            if is_esn==True:
                spectral_hook()

            epoch_loss += loss.item() * data.num_graphs
        
        scheduler.step()  # Step the learning rate scheduler

        epoch_end_time = time.time()
        epoch_time.append(epoch_end_time - epoch_start_time)
        
        # Measure CPU and GPU usage after the epoch
        cpu_usage_after = psutil.cpu_percent(interval=None)
        memory_after = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_after = torch.cuda.memory_allocated(device)
            gpu_util_after = torch.cuda.utilization(device)
        else:
            gpu_usage_after = 0
            gpu_util_after = 0

        # Calculate average CPU and GPU usage during the epoch
        cpu_usage_percent.append((cpu_usage_before + cpu_usage_after) / 2)
        memory_usage.append((memory_before + memory_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage.append((gpu_usage_before + gpu_usage_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage_percent.append((gpu_util_before + gpu_util_after) / 2)

        training_loss.append(epoch_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_data in val_loader:
                x, edge_index, batch, y = val_data.x.to(device), val_data.edge_index.to(device), val_data.batch.to(device), val_data.y.to(device)
                adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)
                val_output = model(x, adj_matrix, batch)
                val_loss += criterion(val_output, y).item() * val_data.num_graphs

        validation_loss.append(val_loss)

        # Early stopping logic considering both training and validation loss
        if val_loss < best_val_loss or epoch_loss < best_train_loss:
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
            epochs_no_improve = 0
            total_epoch = epoch + 1
            torch.save(model.state_dict(), save_path)  # Save the best model
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                total_epoch = epoch + 1
                break

    avg_epoch_time = np.mean(epoch_time)
    avg_cpu_usage_percent = np.mean(cpu_usage_percent)
    avg_memory_usage = np.mean(memory_usage)
    avg_gpu_usage = np.mean(gpu_usage)
    avg_gpu_usage_percent = np.mean(gpu_usage_percent)
    total_training_time = np.sum(epoch_time)
    max_cpu_usage_percent = np.max(cpu_usage_percent)
    max_memory_usage = np.max(memory_usage)
    max_gpu_usage = np.max(gpu_usage)
    max_gpu_usage_percent = np.max(gpu_usage_percent)

    return total_epoch, total_training_time, avg_memory_usage, avg_gpu_usage, max_memory_usage, max_gpu_usage

def binary_evaluation(y_true, y_pred):
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate True Positives, True Negatives, False Positives, False Negatives
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calculate metrics
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    
    return accuracy, sensitivity, specificity

def multiclass_evaluation(y_true, y_pred):
    # Convert to numpy arrays for easier manipulation
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Calculate metrics
    accuracy = np.trace(cm) / np.sum(cm)
    sensitivity = np.zeros(cm.shape[0])
    specificity = np.zeros(cm.shape[0])
    
    for i in range(cm.shape[0]):
        TP = cm[i, i]
        FN = np.sum(cm[i, :]) - TP
        FP = np.sum(cm[:, i]) - TP
        TN = np.sum(cm) - (TP + FN + FP)

        sensitivity[i] = TP / (TP + FN) if (TP + FN) != 0 else 0
        specificity[i] = TN / (TN + FP) if (TN + FP) != 0 else 0

    avg_sensitivity = np.mean(sensitivity)
    avg_specificity = np.mean(specificity)

    return accuracy, avg_sensitivity, avg_specificity

def inference_performance(model, loader):
    model.eval()
    total_inference_time = 0
    cpu_usage_percent = []
    memory_usage = []
    gpu_usage = []
    gpu_usage_percent = []
    labels = []

    # Get the process object for the current process
    process = psutil.Process()

    for data in loader:
        x, edge_index, batch = data.x.to(device), data.edge_index.to(device), data.batch.to(device)
        adj_matrix = utils.to_dense_adj(edge_index).squeeze(0).to(device)
        
        # Measure CPU and GPU usage before inference
        cpu_usage_before = psutil.cpu_percent(interval=None)
        memory_before = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_before = torch.cuda.memory_allocated(device)
            gpu_util_before = torch.cuda.utilization(device)
        else:
            gpu_usage_before = 0
            gpu_util_before = 0

        start_time = time.time()
        with torch.no_grad():
            output = model(x, adj_matrix, batch)
            labels.append(data.y.cpu().numpy())
        end_time = time.time()

        # Measure CPU and GPU usage after inference
        cpu_usage_after = psutil.cpu_percent(interval=None)
        memory_after = process.memory_info().rss
        if torch.cuda.is_available():
            gpu_usage_after = torch.cuda.memory_allocated(device)
            gpu_util_after = torch.cuda.utilization(device)
        else:
            gpu_usage_after = 0
            gpu_util_after = 0

        # Calculate average CPU and GPU usage during inference
        cpu_usage_percent.append((cpu_usage_before + cpu_usage_after) / 2)
        memory_usage.append((memory_before + memory_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage.append((gpu_usage_before + gpu_usage_after) / 2 / (1024**3))  # Convert to GB
        gpu_usage_percent.append((gpu_util_before + gpu_util_after) / 2)

        # Calculate inference time
        inference_time = end_time - start_time
        total_inference_time += inference_time

    avg_cpu_usage_percent = np.mean(cpu_usage_percent)
    avg_memory_usage = np.mean(memory_usage)
    avg_gpu_usage = np.mean(gpu_usage)
    avg_gpu_usage_percent = np.mean(gpu_usage_percent)
    total_inference_time = total_inference_time / len(loader)  # Average inference time per batch

    print(f'\nAverage Inference Time per Batch: {total_inference_time:.4f}s')
    print(f'Average CPU Usage: {avg_cpu_usage_percent:.2f}%')
    print(f'Average Memory Usage: {avg_memory_usage:.2f}GB')
    print(f'Average GPU Usage: {avg_gpu_usage:.2f}GB')
    print(f'Average GPU Utilization: {avg_gpu_usage_percent:.2f}%')

    return 

## Cross Validation
def single_train_test_cv(model, dataset, num_folds=3, lr=0.001, num_epochs=100, patience=10, step_size=50, gamma=0.5, 
                         save_path='models/gcn_x_cv.pth', binary_classification=True, is_esn=False):

    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    fold_results = {
        "accuracy": [],
        "sensitivity": [],
        "specificity": [],
        "epochs": [],
        "training_times": [],
        "testing_times": [],
        "avg_memory_usage": [],
        "avg_gpu_usage": [],
        "max_memory_usage": [],
        "max_gpu_usage": []
    }

    for fold, (train_val_idx, test_idx) in enumerate(kfold.split(dataset)):
        print(f"\nFold {fold+1}/{num_folds}")
        
        set_seed(fold)
        # Create samplers
        train_val_sampler = SubsetRandomSampler(train_val_idx)
        test_sampler = SubsetRandomSampler(test_idx)
        
        # Create data loaders with deterministic behavior
        train_val_loader = DataLoader(dataset, sampler=train_val_sampler, batch_size=32, num_workers=0)
        test_loader = DataLoader(dataset, sampler=test_sampler, batch_size=32, num_workers=0)
        
        # Split train_val indices into train and validation indices
        train_size = int(0.9 * len(train_val_idx))
        val_size = len(train_val_idx) - train_size
        train_indices, val_indices = train_val_idx[:train_size], train_val_idx[train_size:]

        # Create subsets
        train_dataset = Subset(dataset, train_indices)
        val_dataset = Subset(dataset, val_indices)
        
        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=32, worker_init_fn=seed_worker, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=32, worker_init_fn=seed_worker, num_workers=0)
        
        # Initialize model
        model_copy = copy.deepcopy(model)
        
        # Train the model
        # set_seed(42)
        total_epoch, total_training_time, avg_memory_usage, avg_gpu_usage, max_memory_usage, max_gpu_usage = single_train(
            model_copy, train_loader, val_loader, lr=lr, num_epochs=num_epochs, patience=patience, 
            step_size=step_size, gamma=gamma, save_path=save_path, 
            binary_classification=binary_classification, is_esn=is_esn)
        
        # Test the model
        # set_seed(42)
        accuracy, sensitivity, specificity, testing_time = multi_test(model_copy.to(device), test_loader, binary_classification=binary_classification)

        # Collect metrics for the fold
        fold_results["accuracy"].append(accuracy)
        fold_results["sensitivity"].append(sensitivity)
        fold_results["specificity"].append(specificity)
        fold_results["epochs"].append(total_epoch)
        fold_results["training_times"].append(total_training_time)
        fold_results["testing_times"].append(testing_time)
        fold_results["avg_memory_usage"].append(avg_memory_usage)
        fold_results["avg_gpu_usage"].append(avg_gpu_usage)
        fold_results["max_memory_usage"].append(max_memory_usage)
        fold_results["max_gpu_usage"].append(max_gpu_usage)

        print(f"Fold {fold+1} Results:")
        print(f"   Accuracy: {accuracy:.4f}")
        print(f"   Average Sensitivity (Recall): {sensitivity:.4f}")
        print(f"   Average Specificity: {specificity:.4f}")

    # Compute average metrics across all folds
    avg_metrics = {metric: np.mean(fold_results[metric]) for metric in fold_results}
    std_metrics = {metric: np.std(fold_results[metric]) for metric in fold_results}

    print("\nCross-Validation Results:")
    for metric in avg_metrics:
        avg_value = avg_metrics[metric]
        std_value = std_metrics[metric]
        print(f"{metric}: Mean = {avg_value:.4f} ± {std_value:.2f}")

    return avg_metrics


## Simulated Dataset

In [29]:
set_seed(42)
gcn_simulated_ones = GCN2Layer(simulated_ones_num_features, 2*simulated_ones_num_features, simulated_ones_num_classes).to(device)
print(gcn_simulated_ones)
print(f"Total number of trainable parameters: {(gcn_simulated_ones.count_parameters())}\n")
avg_metrics = single_train_test_cv(gcn_simulated_ones, simulated_ones, 
            lr=0.005, num_epochs=500, step_size=500,  
            save_path='models/gcn_simulated_ones.pth', binary_classification=False)

GCN2Layer(
  (gcn1): GCN (8 -> 16)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gcn2): GCN (16 -> 32)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)
Total number of trainable parameters: 883


Fold 1/3
Fold 1 Results:
   Accuracy: 0.2900
   Average Sensitivity (Recall): 0.3333
   Average Specificity: 0.6667

Fold 2/3
Fold 2 Results:
   Accuracy: 0.3600
   Average Sensitivity (Recall): 0.3411
   Average Specificity: 0.6703

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4400
   Average Sensitivity (Recall): 0.4353
   Average Specificity: 0.7177

Cross-Validation Results:
accuracy: Mean = 0.3633 ± 0.06
sensitivity: Mean = 0.3699 ± 0.05
specificity: Mean = 0.6849 ± 0.02
epochs: Mean = 325.0000 ± 70.60
training_times: Mean = 9.5425 ± 2.05
testing_times: Mean = 0.0111 ± 0.00
avg_memory_usage: Mean = 0.1657 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
ma

In [30]:
set_seed(42)
gcn_simulated_random = GCN2Layer(simulated_random_num_features, 2*simulated_random_num_features, simulated_random_num_classes).to(device)
print(gcn_simulated_random)
print(f"Total number of trainable parameters: {(gcn_simulated_random.count_parameters())}\n")
avg_metrics = single_train_test_cv(gcn_simulated_random, simulated_random, 
            lr=0.005, num_epochs=500, step_size=500,  
            save_path='models/gcn_simulated_random.pth', binary_classification=False)

GCN2Layer(
  (gcn1): GCN (8 -> 16)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gcn2): GCN (16 -> 32)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)
Total number of trainable parameters: 883


Fold 1/3
Fold 1 Results:
   Accuracy: 0.3400
   Average Sensitivity (Recall): 0.3599
   Average Specificity: 0.6799

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4000
   Average Sensitivity (Recall): 0.4269
   Average Specificity: 0.7102

Fold 3/3
Fold 3 Results:
   Accuracy: 0.3700
   Average Sensitivity (Recall): 0.3671
   Average Specificity: 0.6857

Cross-Validation Results:
accuracy: Mean = 0.3700 ± 0.02
sensitivity: Mean = 0.3846 ± 0.03
specificity: Mean = 0.6919 ± 0.01
epochs: Mean = 217.3333 ± 199.92
training_times: Mean = 6.4085 ± 5.79
testing_times: Mean = 0.0115 ± 0.00
avg_memory_usage: Mean = 0.1430 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
m

In [32]:
set_seed(42)
gcn_simulated_laplacian = GCN2Layer(simulated_laplacian_num_features, 2*simulated_laplacian_num_features, simulated_laplacian_num_classes).to(device)
print(gcn_simulated_laplacian)
print(f"Total number of trainable parameters: {(gcn_simulated_laplacian.count_parameters())}\n")
avg_metrics = single_train_test_cv(gcn_simulated_laplacian, simulated_laplacian, 
            lr=0.005, num_epochs=500, step_size=500,  
            save_path='models/gcn_simulated_laplacian.pth', binary_classification=False)

GCN2Layer(
  (gcn1): GCN (8 -> 16)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gcn2): GCN (16 -> 32)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32, out_features=3, bias=True)
)
Total number of trainable parameters: 883


Fold 1/3
Fold 1 Results:
   Accuracy: 0.3600
   Average Sensitivity (Recall): 0.3425
   Average Specificity: 0.6677

Fold 2/3
Fold 2 Results:
   Accuracy: 0.3400
   Average Sensitivity (Recall): 0.3512
   Average Specificity: 0.6740

Fold 3/3
Fold 3 Results:
   Accuracy: 0.3800
   Average Sensitivity (Recall): 0.3683
   Average Specificity: 0.6842

Cross-Validation Results:
accuracy: Mean = 0.3600 ± 0.02
sensitivity: Mean = 0.3540 ± 0.01
specificity: Mean = 0.6753 ± 0.01
epochs: Mean = 223.0000 ± 195.94
training_times: Mean = 6.4610 ± 5.66
testing_times: Mean = 0.0100 ± 0.00
avg_memory_usage: Mean = 0.1524 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
m

## EMCI-AD Dataset

In [39]:
set_seed(42)
gcn_emci_ones = GCN2Layer(emci_ones_num_features, 2*emci_ones_num_features, emci_ones_num_classes).to(device)
print(gcn_emci_ones)
print(f"Total number of trainable parameters: {(gcn_emci_ones.count_parameters())}\n")
avg_metrics = single_train_test_cv(gcn_emci_ones, emci_ones, 
            lr=0.005, num_epochs=500, step_size=500,  
            save_path='models/gcn_emci_ones.pth', binary_classification=True)

GCN2Layer(
  (gcn1): GCN (8 -> 16)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gcn2): GCN (16 -> 32)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)
Total number of trainable parameters: 850


Fold 1/3
Fold 1 Results:
   Accuracy: 0.5111
   Average Sensitivity (Recall): 0.0000
   Average Specificity: 1.0000

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.0000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.4318
   Average Sensitivity (Recall): 0.0000
   Average Specificity: 1.0000

Cross-Validation Results:
accuracy: Mean = 0.4625 ± 0.03
sensitivity: Mean = 0.3333 ± 0.47
specificity: Mean = 0.6667 ± 0.47
epochs: Mean = 24.6667 ± 7.32
training_times: Mean = 0.3475 ± 0.10
testing_times: Mean = 0.0050 ± 0.00
avg_memory_usage: Mean = 0.1657 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_

In [38]:
set_seed(42)
gcn_emci_random = GCN2Layer(emci_random_num_features, 2*emci_random_num_features, emci_random_num_classes).to(device)
print(gcn_emci_random)
print(f"Total number of trainable parameters: {(gcn_emci_random.count_parameters())}\n")
avg_metrics = single_train_test_cv(gcn_emci_random, emci_random, 
            lr=0.005, num_epochs=500, step_size=500,  
            save_path='models/gcn_emci_random.pth', binary_classification=True)

GCN2Layer(
  (gcn1): GCN (8 -> 16)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gcn2): GCN (16 -> 32)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)
Total number of trainable parameters: 850


Fold 1/3
Fold 1 Results:
   Accuracy: 0.5556
   Average Sensitivity (Recall): 0.5455
   Average Specificity: 0.5652

Fold 2/3
Fold 2 Results:
   Accuracy: 0.5778
   Average Sensitivity (Recall): 0.8500
   Average Specificity: 0.3600

Fold 3/3
Fold 3 Results:
   Accuracy: 0.6364
   Average Sensitivity (Recall): 0.4800
   Average Specificity: 0.8421

Cross-Validation Results:
accuracy: Mean = 0.5899 ± 0.03
sensitivity: Mean = 0.6252 ± 0.16
specificity: Mean = 0.5891 ± 0.20
epochs: Mean = 500.0000 ± 0.00
training_times: Mean = 6.7980 ± 0.19
testing_times: Mean = 0.0046 ± 0.00
avg_memory_usage: Mean = 0.1648 ± 0.00
avg_gpu_usage: Mean = 0.0000 ± 0.00
max

In [37]:
set_seed(42)
gcn_emci_laplacian = GCN2Layer(emci_laplacian_num_features, 2*emci_laplacian_num_features, emci_laplacian_num_classes).to(device)
print(gcn_emci_laplacian)
print(f"Total number of trainable parameters: {(gcn_emci_laplacian.count_parameters())}\n")
avg_metrics = single_train_test_cv(gcn_emci_laplacian, emci_laplacian, 
            lr=0.005, num_epochs=500, step_size=500,  
            save_path='models/gcn_emci_laplacian.pth', binary_classification=True)

GCN2Layer(
  (gcn1): GCN (8 -> 16)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gcn2): GCN (16 -> 32)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)
Total number of trainable parameters: 850


Fold 1/3
Fold 1 Results:
   Accuracy: 0.4889
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.0000

Fold 2/3
Fold 2 Results:
   Accuracy: 0.4444
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.0000

Fold 3/3
Fold 3 Results:
   Accuracy: 0.5682
   Average Sensitivity (Recall): 1.0000
   Average Specificity: 0.0000

Cross-Validation Results:
accuracy: Mean = 0.5005 ± 0.05
sensitivity: Mean = 1.0000 ± 0.00
specificity: Mean = 0.0000 ± 0.00
epochs: Mean = 72.3333 ± 5.31
training_times: Mean = 1.0024 ± 0.05
testing_times: Mean = 0.0046 ± 0.00
avg_memory_usage: Mean = 0.3332 ± 0.01
avg_gpu_usage: Mean = 0.0000 ± 0.00
max_